# Smart Fertilizer Optimizer Training Notebook

This notebook demonstrates the training and optimization of fertilizer recommendation algorithms using African agricultural data.

## Objectives
1. Train machine learning models for yield prediction
2. Optimize fertilizer recommendation algorithms
3. Validate STCR coefficients for African conditions
4. Analyze regional fertilizer efficiency patterns

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")

print("Smart Fertilizer Optimizer Training")
print("===================================")

## 1. Data Loading and Exploration

In [ ]:
# Load training data
yield_data = pd.read_csv('../data/yield_training_data.csv')
soil_data = pd.read_csv('../data/soil_samples.csv')

print("Yield Training Data Shape:", yield_data.shape)
print("Soil Samples Data Shape:", soil_data.shape)

# Display basic statistics
print("\nYield Data Summary:")
print(yield_data.describe())

print("\nCrop Distribution:")
print(yield_data['crop'].value_counts())

print("\nRegion Distribution:")
print(yield_data['region'].value_counts())

In [ ]:
# Visualization of yield distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Yield by crop
axes[0,0].boxplot([yield_data[yield_data['crop'] == crop]['yield_tons_per_ha'].values 
                   for crop in yield_data['crop'].unique()], 
                  labels=yield_data['crop'].unique())
axes[0,0].set_title('Yield Distribution by Crop')
axes[0,0].set_ylabel('Yield (tons/ha)')
axes[0,0].tick_params(axis='x', rotation=45)

# Yield by region
yield_data.groupby('region')['yield_tons_per_ha'].mean().plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Average Yield by Region')
axes[0,1].set_ylabel('Yield (tons/ha)')
axes[0,1].tick_params(axis='x', rotation=45)

# Fertilizer application vs yield
axes[1,0].scatter(yield_data['nitrogen_applied'], yield_data['yield_tons_per_ha'], alpha=0.6)
axes[1,0].set_xlabel('Nitrogen Applied (kg/ha)')
axes[1,0].set_ylabel('Yield (tons/ha)')
axes[1,0].set_title('Nitrogen Application vs Yield')

# Soil pH vs yield
axes[1,1].scatter(yield_data['soil_ph'], yield_data['yield_tons_per_ha'], alpha=0.6)
axes[1,1].set_xlabel('Soil pH')
axes[1,1].set_ylabel('Yield (tons/ha)')
axes[1,1].set_title('Soil pH vs Yield')

plt.tight_layout()
plt.show()

## 2. Feature Engineering and Data Preparation

In [ ]:
# Feature engineering
def engineer_features(df):
    """
    Create additional features for model training
    """
    df_processed = df.copy()
    
    # Convert categorical variables
    le_region = LabelEncoder()
    le_crop = LabelEncoder()
    le_country = LabelEncoder()
    
    df_processed['region_encoded'] = le_region.fit_transform(df_processed['region'])
    df_processed['crop_encoded'] = le_crop.fit_transform(df_processed['crop'])
    df_processed['country_encoded'] = le_country.fit_transform(df_processed['country'])
    
    # Create interaction features
    df_processed['npk_total'] = (df_processed['nitrogen_applied'] + 
                                 df_processed['phosphorus_applied'] + 
                                 df_processed['potassium_applied'])
    
    df_processed['nutrient_balance'] = (df_processed['nitrogen_applied'] / 
                                        (df_processed['phosphorus_applied'] + 1))
    
    # Soil health index
    df_processed['soil_health_index'] = (df_processed['organic_matter'] * 0.4 + 
                                         (7.0 - abs(df_processed['soil_ph'] - 6.5)) * 0.6)
    
    # Climate suitability index
    df_processed['climate_index'] = df_processed['rainfall_mm'] / 1000 * df_processed['temperature_avg'] / 30
    
    # Management factor
    management_map = {'low': 1, 'medium': 2, 'high': 3}
    df_processed['mechanization_numeric'] = df_processed['mechanization_level'].map(management_map)
    
    pressure_map = {'low': 1, 'medium': 2, 'high': 3}
    df_processed['pest_pressure_numeric'] = df_processed['pest_pressure'].map(pressure_map)
    df_processed['disease_pressure_numeric'] = df_processed['disease_pressure'].map(pressure_map)
    
    fertility_map = {'very_poor': 1, 'poor': 2, 'fair': 3, 'good': 4, 'excellent': 5}
    df_processed['soil_fertility_numeric'] = df_processed['soil_fertility_rating'].map(fertility_map)
    
    return df_processed

# Apply feature engineering
yield_data_processed = engineer_features(yield_data)

print("Feature Engineering Complete")
print("New features created:")
print("- NPK Total")
print("- Nutrient Balance")
print("- Soil Health Index")
print("- Climate Index")
print("- Management Factors (numeric)")

## 3. STCR Coefficient Optimization

In [ ]:
def stcr_yield_function(params, soil_n, soil_p, soil_k, fert_n, fert_p, fert_k):
    """
    STCR yield prediction function
    Y = a + b1*SN + b2*SP + b3*SK + c1*FN + c2*FP + c3*FK
    """
    a, b1, b2, b3, c1, c2, c3 = params
    return a + b1*soil_n + b2*soil_p + b3*soil_k + c1*fert_n + c2*fert_p + c3*fert_k

def optimize_stcr_coefficients(crop_data):
    """
    Optimize STCR coefficients for a specific crop
    """
    def objective(params):
        predicted = stcr_yield_function(
            params,
            crop_data['soil_n'].values,
            crop_data['soil_p'].values, 
            crop_data['soil_k'].values,
            crop_data['nitrogen_applied'].values,
            crop_data['phosphorus_applied'].values,
            crop_data['potassium_applied'].values
        )
        return np.mean((predicted - crop_data['yield_tons_per_ha'].values)**2)
    
    # Initial guess
    initial_params = [2.0, 0.01, 0.05, 0.01, 0.02, 0.08, 0.01]
    
    # Optimization bounds
    bounds = [(0, 10), (0, 0.1), (0, 0.2), (0, 0.1), (0, 0.1), (0, 0.2), (0, 0.1)]
    
    result = minimize(objective, initial_params, bounds=bounds, method='L-BFGS-B')
    return result.x

# Prepare soil nutrient data (convert from ppm to kg/ha)
conversion_factor = 2.2  # Assuming 20cm depth, 2.2 million kg soil per hectare

yield_data_processed['soil_n'] = yield_data_processed['nitrogen_applied'] * 0.6  # Estimate from applied
yield_data_processed['soil_p'] = yield_data_processed['phosphorus_applied'] * 0.3  # Estimate from applied 
yield_data_processed['soil_k'] = yield_data_processed['potassium_applied'] * 0.5  # Estimate from applied

# Optimize STCR coefficients by crop
stcr_coefficients = {}
crops_to_optimize = ['maize', 'rice', 'wheat', 'sorghum']

for crop in crops_to_optimize:
    if crop in yield_data_processed['crop'].values:
        crop_data = yield_data_processed[yield_data_processed['crop'] == crop]
        if len(crop_data) >= 5:  # Minimum data points needed
            coeffs = optimize_stcr_coefficients(crop_data)
            stcr_coefficients[crop] = {
                'a': coeffs[0],
                'b1': coeffs[1], 'b2': coeffs[2], 'b3': coeffs[3],
                'c1': coeffs[4], 'c2': coeffs[5], 'c3': coeffs[6]
            }
            print(f"\n{crop.upper()} STCR Coefficients:")
            print(f"a (base yield): {coeffs[0]:.3f}")
            print(f"Soil coefficients - N: {coeffs[1]:.4f}, P: {coeffs[2]:.4f}, K: {coeffs[3]:.4f}")
            print(f"Fertilizer coefficients - N: {coeffs[4]:.4f}, P: {coeffs[5]:.4f}, K: {coeffs[6]:.4f}")

print("\nSTCR Coefficient Optimization Complete")

## 4. Machine Learning Model Training

In [ ]:
# Select features for modeling
feature_columns = [
    'region_encoded', 'crop_encoded', 'area_hectares',
    'rainfall_mm', 'temperature_avg', 'soil_ph', 'organic_matter',
    'nitrogen_applied', 'phosphorus_applied', 'potassium_applied',
    'farmer_experience_years', 'mechanization_numeric', 'market_distance_km',
    'pest_pressure_numeric', 'disease_pressure_numeric', 'soil_fertility_numeric',
    'npk_total', 'nutrient_balance', 'soil_health_index', 'climate_index'
]

# Prepare data
X = yield_data_processed[feature_columns].fillna(0)
y = yield_data_processed['yield_tons_per_ha']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")

In [ ]:
# Train multiple models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

model_results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train model
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    model_results[name] = {
        'model': model,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"RMSE: {rmse:.3f}")
    print(f"MAE: {mae:.3f}")
    print(f"R²: {r2:.3f}")

# Find best model
best_model_name = min(model_results.keys(), key=lambda x: model_results[x]['rmse'])
print(f"\nBest Model: {best_model_name}")
print(f"Best RMSE: {model_results[best_model_name]['rmse']:.3f}")

## 5. Feature Importance Analysis

In [ ]:
# Feature importance for Random Forest
rf_model = model_results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
plt.title('Top 15 Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("Top 10 Most Important Features:")
print(feature_importance.head(10))

## 6. Model Validation and Predictions

In [ ]:
# Plot actual vs predicted for best model
best_predictions = model_results[best_model_name]['predictions']

plt.figure(figsize=(10, 8))
plt.scatter(y_test, best_predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Yield (tons/ha)')
plt.ylabel('Predicted Yield (tons/ha)')
plt.title(f'Actual vs Predicted Yield ({best_model_name})')

# Add R² to plot
r2_best = model_results[best_model_name]['r2']
plt.text(0.05, 0.95, f'R² = {r2_best:.3f}', transform=plt.gca().transAxes, 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Residual analysis
residuals = y_test - best_predictions

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(best_predictions, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Yield (tons/ha)')
plt.ylabel('Residuals')
plt.title('Residual Plot')

plt.subplot(1, 2, 2)
plt.hist(residuals, bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residual Distribution')

plt.tight_layout()
plt.show()

## 7. Fertilizer Efficiency Analysis

In [ ]:
# Calculate fertilizer use efficiency
def calculate_fertilizer_efficiency(data):
    """
    Calculate agronomic efficiency (AE) and partial factor productivity (PFP)
    """
    data = data.copy()
    
    # Assume control yield (without fertilizer) is 70% of actual yield
    data['control_yield'] = data['yield_tons_per_ha'] * 0.7
    
    # Agronomic Efficiency (kg grain per kg nutrient)
    data['ae_n'] = (data['yield_tons_per_ha'] - data['control_yield']) / (data['nitrogen_applied'] / 1000)
    data['ae_p'] = (data['yield_tons_per_ha'] - data['control_yield']) / (data['phosphorus_applied'] / 1000)
    data['ae_k'] = (data['yield_tons_per_ha'] - data['control_yield']) / (data['potassium_applied'] / 1000)
    
    # Partial Factor Productivity (kg grain per kg nutrient)
    data['pfp_n'] = data['yield_tons_per_ha'] / (data['nitrogen_applied'] / 1000)
    data['pfp_p'] = data['yield_tons_per_ha'] / (data['phosphorus_applied'] / 1000)
    data['pfp_k'] = data['yield_tons_per_ha'] / (data['potassium_applied'] / 1000)
    
    return data

# Calculate efficiency metrics
efficiency_data = calculate_fertilizer_efficiency(yield_data_processed)

# Remove infinite values
efficiency_columns = ['ae_n', 'ae_p', 'ae_k', 'pfp_n', 'pfp_p', 'pfp_k']
for col in efficiency_columns:
    efficiency_data[col] = efficiency_data[col].replace([np.inf, -np.inf], np.nan)

# Summary by crop
efficiency_summary = efficiency_data.groupby('crop')[efficiency_columns].mean()
print("Fertilizer Efficiency by Crop:")
print(efficiency_summary.round(2))

# Plot efficiency metrics
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

crops = efficiency_data['crop'].unique()
metrics = ['ae_n', 'ae_p', 'ae_k', 'pfp_n', 'pfp_p', 'pfp_k']
titles = ['Agronomic Efficiency - N', 'Agronomic Efficiency - P', 'Agronomic Efficiency - K',
          'Partial Factor Productivity - N', 'Partial Factor Productivity - P', 'Partial Factor Productivity - K']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    row, col = i // 3, i % 3
    
    crop_data = []
    crop_labels = []
    
    for crop in crops:
        crop_values = efficiency_data[efficiency_data['crop'] == crop][metric].dropna()
        if len(crop_values) > 0:
            crop_data.append(crop_values.values)
            crop_labels.append(crop)
    
    if crop_data:
        axes[row, col].boxplot(crop_data, labels=crop_labels)
        axes[row, col].set_title(title)
        axes[row, col].tick_params(axis='x', rotation=45)
        axes[row, col].set_ylabel('Efficiency (kg/kg)')

plt.tight_layout()
plt.show()

## 8. Regional Analysis

In [ ]:
# Regional yield and fertilizer use analysis
regional_analysis = yield_data_processed.groupby('region').agg({
    'yield_tons_per_ha': ['mean', 'std'],
    'nitrogen_applied': 'mean',
    'phosphorus_applied': 'mean', 
    'potassium_applied': 'mean',
    'soil_ph': 'mean',
    'organic_matter': 'mean',
    'rainfall_mm': 'mean',
    'temperature_avg': 'mean'
}).round(2)

print("Regional Analysis Summary:")
print(regional_analysis)

# Plot regional comparisons
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Average yield by region
regional_yield = yield_data_processed.groupby('region')['yield_tons_per_ha'].mean()
regional_yield.plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Average Yield by Region')
axes[0,0].set_ylabel('Yield (tons/ha)')
axes[0,0].tick_params(axis='x', rotation=45)

# Fertilizer use by region
regional_fert = yield_data_processed.groupby('region')[['nitrogen_applied', 'phosphorus_applied', 'potassium_applied']].mean()
regional_fert.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Average Fertilizer Use by Region')
axes[0,1].set_ylabel('Fertilizer Applied (kg/ha)')
axes[0,1].tick_params(axis='x', rotation=45)

# Soil health by region
regional_soil = yield_data_processed.groupby('region')[['soil_ph', 'organic_matter']].mean()
regional_soil.plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Soil Conditions by Region')
axes[1,0].tick_params(axis='x', rotation=45)

# Climate by region
regional_climate = yield_data_processed.groupby('region')[['rainfall_mm', 'temperature_avg']].mean()
ax2 = axes[1,1]
ax3 = ax2.twinx()

regional_climate['rainfall_mm'].plot(kind='bar', ax=ax2, color='blue', alpha=0.7, label='Rainfall')
regional_climate['temperature_avg'].plot(kind='bar', ax=ax3, color='red', alpha=0.7, label='Temperature')

ax2.set_ylabel('Rainfall (mm)', color='blue')
ax3.set_ylabel('Temperature (°C)', color='red')
ax2.set_title('Climate Conditions by Region')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. Optimization Recommendations

In [ ]:
# Generate optimization recommendations
def generate_optimization_recommendations(model_results, efficiency_data, regional_analysis):
    """
    Generate recommendations for improving fertilizer use efficiency
    """
    recommendations = {
        'model_performance': {},
        'efficiency_insights': {},
        'regional_priorities': {},
        'implementation_suggestions': []
    }
    
    # Model performance insights
    best_model = min(model_results.keys(), key=lambda x: model_results[x]['rmse'])
    recommendations['model_performance'] = {
        'best_model': best_model,
        'rmse': model_results[best_model]['rmse'],
        'r2': model_results[best_model]['r2']
    }
    
    # Efficiency insights
    avg_efficiency = efficiency_data[['ae_n', 'ae_p', 'ae_k']].mean()
    recommendations['efficiency_insights'] = {
        'nitrogen_efficiency': avg_efficiency['ae_n'],
        'phosphorus_efficiency': avg_efficiency['ae_p'],
        'potassium_efficiency': avg_efficiency['ae_k']
    }
    
    # Regional priorities
    lowest_yield_region = regional_analysis[('yield_tons_per_ha', 'mean')].idxmin()
    highest_yield_region = regional_analysis[('yield_tons_per_ha', 'mean')].idxmax()
    
    recommendations['regional_priorities'] = {
        'focus_region': lowest_yield_region,
        'model_region': highest_yield_region,
        'yield_gap': (regional_analysis.loc[highest_yield_region, ('yield_tons_per_ha', 'mean')] - 
                     regional_analysis.loc[lowest_yield_region, ('yield_tons_per_ha', 'mean')])
    }
    
    # Implementation suggestions
    suggestions = [
        "Implement site-specific fertilizer recommendations using the trained ML model",
        "Focus on improving soil organic matter in low-yield regions",
        "Optimize nitrogen timing and splitting for better efficiency",
        "Develop region-specific STCR coefficients for local conditions",
        "Integrate weather forecasting for application timing",
        "Promote balanced NPK fertilization based on soil testing",
        "Establish demonstration plots in priority regions",
        "Train farmers on precision fertilizer application techniques"
    ]
    recommendations['implementation_suggestions'] = suggestions
    
    return recommendations

# Generate recommendations
optimization_recommendations = generate_optimization_recommendations(
    model_results, efficiency_data, regional_analysis
)

print("OPTIMIZATION RECOMMENDATIONS")
print("="*50)

print("\n1. MODEL PERFORMANCE:")
print(f"   Best Model: {optimization_recommendations['model_performance']['best_model']}")
print(f"   RMSE: {optimization_recommendations['model_performance']['rmse']:.3f}")
print(f"   R²: {optimization_recommendations['model_performance']['r2']:.3f}")

print("\n2. EFFICIENCY INSIGHTS:")
print(f"   Nitrogen Efficiency: {optimization_recommendations['efficiency_insights']['nitrogen_efficiency']:.2f} kg/kg")
print(f"   Phosphorus Efficiency: {optimization_recommendations['efficiency_insights']['phosphorus_efficiency']:.2f} kg/kg")
print(f"   Potassium Efficiency: {optimization_recommendations['efficiency_insights']['potassium_efficiency']:.2f} kg/kg")

print("\n3. REGIONAL PRIORITIES:")
print(f"   Focus Region: {optimization_recommendations['regional_priorities']['focus_region']}")
print(f"   Model Region: {optimization_recommendations['regional_priorities']['model_region']}")
print(f"   Yield Gap: {optimization_recommendations['regional_priorities']['yield_gap']:.2f} tons/ha")

print("\n4. IMPLEMENTATION SUGGESTIONS:")
for i, suggestion in enumerate(optimization_recommendations['implementation_suggestions'], 1):
    print(f"   {i}. {suggestion}")


## 10. Model Export and Summary

In [ ]:
# Save trained models and coefficients
import pickle
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save best model
best_model = model_results[best_model_name]['model']
with open(f'../models/best_yield_model_{best_model_name.lower().replace(" ", "_")}.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save scaler
with open('../models/feature_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save STCR coefficients
with open('../models/stcr_coefficients.pkl', 'wb') as f:
    pickle.dump(stcr_coefficients, f)

# Save feature names
with open('../models/feature_columns.pkl', 'wb') as f:
    pickle.dump(feature_columns, f)

print("Models and artifacts saved successfully!")
print("\nSUMMARY OF TRAINING SESSION")
print("="*40)
print(f"Training Data Points: {len(yield_data_processed)}")
print(f"Number of Features: {len(feature_columns)}")
print(f"Best Model: {best_model_name}")
print(f"Best RMSE: {model_results[best_model_name]['rmse']:.3f}")
print(f"Best R²: {model_results[best_model_name]['r2']:.3f}")
print(f"Crops Analyzed: {len(yield_data_processed['crop'].unique())}")
print(f"Regions Covered: {len(yield_data_processed['region'].unique())}")
print(f"STCR Coefficients Generated: {len(stcr_coefficients)}")

print("\nNext Steps:")
print("1. Integrate trained models into the Smart Fertilizer application")
print("2. Validate recommendations with field trials")
print("3. Collect more data for continuous model improvement")
print("4. Implement feedback mechanisms for model refinement")
